In [2]:
import numpy as np
import copy

In [3]:
n = 100
k = 6
S = np.random.randint(1, 100, n)

In [4]:
def cur_of(ms, mat):
    _ideal = np.sum(ms) / mat.shape[1]
    return np.sum((ms @ mat - _ideal)**2)

In [5]:
def cur_of_from_sums(ms, sums):
    _ideal = np.sum(ms) / len(sums)
    return np.sum((sums - _ideal)**2)

In [6]:
def sol_list_to_matrix(sol_list, k):
    if len(np.shape(sol_list)) > 1 and np.shape(sol_list)[1] > 1:
        return sol_list
    else:
        cur_sol1 = np.zeros((sol_list.shape[0], k))
        for pos1, pos2 in enumerate(sol_list):
            cur_sol1[pos1, pos2] = 1.
        return cur_sol1

In [7]:
def change_sol_to_other_random(cur_sol, n=None, k=None, max_depth=1):
    _cur_sol = copy.deepcopy(cur_sol)
    _rs = np.random.RandomState()
    if max_depth == 1:
        depth = 1
    else:
        depth = _rs.randint(1, max_depth)
    for d in range(depth):
        i = _rs.randint(n)
        pos1 = np.argmax(_cur_sol[i])
        pos2 = _rs.randint(k)
        while pos2 == pos1:
            pos2 = _rs.randint(k)
        _cur_sol[i, pos1] = 0.
        _cur_sol[i, pos2] = 1.
    return _cur_sol

In [8]:
def generate_mnp_change(cur_sol, n=None, k=None, min_depth=1, max_depth=1):
    _rs = np.random.RandomState()
    depth = _rs.randint(min_depth, max_depth+1)
    change = []
    for d in range(depth):
        i = _rs.randint(n)
        pos1 = cur_sol[i].argmax()
        pos2 = _rs.randint(k)
        while pos2 == pos1:
            pos2 = _rs.randint(k)
        change.append([i, pos1, pos2])
    return change

In [9]:
def make_mnp_change(cur_sol, change):
    _cur_sol = cur_sol
    for ch_part in change:
        _cur_sol[ch_part[0], ch_part[1]] = 0.
        _cur_sol[ch_part[0], ch_part[2]] = 1.
    return _cur_sol

In [10]:
def generate_good_start_sol(ms, k, of_limit, start_i=0, max_iters=1000):
    i = start_i
    while i <= max_iters:
        rs = np.random.RandomState(i)
        cur_sol = rs.randint(0, k, ms.shape[0])
        cur_sol = sol_list_to_matrix(cur_sol, k)
        cur_o_f = cur_of(ms, cur_sol)
        if cur_o_f < of_limit:
            return [i, cur_sol, cur_o_f]
        else:
            i += 1
    return None

In [11]:
def hill_climbing(ms, k, max_iters=100, min_depth=1, max_depth=1):
    _rs = np.random.RandomState()
    n = len(ms)
    cur_sol = _rs.randint(0, k, ms.shape[0])

    cur_sol = sol_list_to_matrix(cur_sol, k)

    subs_sums = ms @ cur_sol
    cur_o_f = cur_of_from_sums(ms, subs_sums)
    iters = 0
    while iters < max_iters:
        iters += 1
        change = generate_mnp_change(cur_sol, n=n, k=k, min_depth=min_depth, max_depth=max_depth)
        new_sums = ms @ cur_sol
        for ch_part in change:
            new_sums[ch_part[1]] -= ms[ch_part[0]]
            new_sums[ch_part[2]] += ms[ch_part[0]]
        new_o_f = cur_of_from_sums(ms, new_sums)
        if new_o_f < cur_o_f:
            make_mnp_change(cur_sol, change)
            cur_o_f = new_o_f
        if new_o_f == 0:
            break
    return cur_sol

In [16]:
h = hill_climbing(S,k)

In [17]:
S@h

array([850., 834., 830., 864., 848., 853.])